In [1]:
import re
import csv
import nltk
import json
import time
import pickle
import tweepy
import pandas as pd
import nltk.classify
from tweepy import OAuthHandler
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.tokenize import RegexpTokenizer
from tweepy.streaming import StreamListener
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
def removeTagsAndLinks(tweet):
    cleaned_tweet=re.sub("@[A-Za-z0-9]+|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",tweet)
    return cleaned_tweet

In [3]:
def tokenise(tweet):
    cleaned_tweet=removeTagsAndLinks(tweet)
    sentences=sent_tokenize(cleaned_tweet)
    word=RegexpTokenizer(r'\w+')
    words=[word.tokenize(sentence) for sentence in sentences]
    return words

In [4]:
def replaceTwoOrMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1\1", s)

def processTweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    tweet = re.sub('@[^\s]+','AT_USER',tweet)    
    tweet = re.sub('[\s]+', ' ', tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = tweet.strip('\'"')
    return tweet


def getStopWordList(stopWordListFileName):
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open(r'stopwordsID.txt', 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords


def getFeatureVector(tweet, stopWords):
    featureVector = []  
    words = tweet.split()
    for w in words:
        w = replaceTwoOrMore(w) 
        w = w.strip('\'"?,.')
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*[a-zA-Z]+[a-zA-Z0-9]*$", w)
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector    

def removeStopwords(tweet):
    stop_words=set(stopwords.words('indonesian'))
    filtered_tweet=[word for dummytweet in tweet for word in  dummytweet  if word not in stop_words]
    return filtered_tweet

In [5]:
def convert(tweet):
    converted_tweet=''
    for word in tweet:
        converted_tweet+=word+" "
    return converted_tweet


In [6]:
def authorize():
    cunsomer_key = 'd3GCOcnCQVrHqYdbkA6p5B0sb'
    consumer_secret = '3XqfMNq4Aj6rnuUjjV6ZCiQdt4SPeWG6N5l5CcrXf1C540vpnV'
    access_token = '772637934390345728-ZtWEKm6pab38OFDtXqlRgjcEvmdRZ9v'
    access_token_secret = 'MHx06gmb5SC4EeXIkvJFKMwyxZD3PMDBOGoGyH6j6fC6A'
    auth = tweepy.OAuthHandler(cunsomer_key , consumer_secret)
    auth.set_access_token(access_token,access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api

In [9]:
def analyze(query,limit=100000,language='in'):
    list_tweet = []
    list_kelas = []
    summary={'Positive':0,'Negative':0,'Neutral':0}
    api=authorize()
    sid=SentimentIntensityAnalyzer()
    
    tweets=tweepy.Cursor(api.search,q=query,lang=language).items(limit)
        
    for tweet in tweets:
        tweet_text= tweet.text
        tweet=tokenise(tweet_text)
        tweet=removeStopwords(tweet)
        tweet=convert(tweet)
        score=sid.polarity_scores(tweet)
        sentiment = ''
        if score['compound']==0:
            summary['Neutral']+=1
            sentiment = 'Neutral'
        elif score['compound']>0:
            summary['Positive']+=1
            sentiment = 'Positive'
        else:
            summary['Negative']+=1
            sentiment = 'Negative'
        list_tweet.append(tweet)
        list_kelas.append(sentiment)
    for keys in summary.keys():
        print("No of %s tweets are %d"%(keys,summary[keys]))
    df = pd.DataFrame({'tweet': list_tweet,'kelas': list_kelas})
    df.to_csv('data_crawling.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
def main():
    query=input('Enter the key to be searched:')
    no=int(input('Enter the number of tweets to be analyzed:'))
    try:
        language=input('Enter the language:')
    except:
        language='in'
    print('====================================================')
    analyze(query,no,language)
if __name__=='__main__':
    main()

In [35]:
list_data = []
reader = csv.reader(open("datatrain.csv"))
stopWords = getStopWordList('stopwordsID.txt')
total = 0
for index,raw in enumerate(reader):
    if index == 0:
        pass
    else:
        sentiment = raw[1]
        tweet = raw[2]
        tweet = processTweet(tweet)
        tweet = removeTagsAndLinks(tweet)
        tweet = getFeatureVector(tweet,stopWords)
        tweet = ' '.join(tweet)
        mydata = (tweet,sentiment)
        list_data.append(mydata)
        total += 1

print(total)
json_dict = {}
all_words = set(word.lower() for passage in list_data for word in word_tokenize(passage[0]))
json_dict['corpus'] = [word for word in all_words] 

with open('corpus.json', 'w') as outfile:
    json.dump(json_dict, outfile)

t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1]) for x in list_data]

classifier = nltk.NaiveBayesClassifier.train(t)

f = open('sentiment.pickle', 'wb')
pickle.dump(classifier, f)
f.close()

23899


In [16]:
stopWords = getStopWordList('stopwordsID.txt')
f = open('sentiment.pickle', 'rb')
classifier = pickle.load(f)
f.close()

d = open('corpus.json')
all_words = json.load(d)
d.close
all_words = all_words['corpus']

list_tweet =  []
list_kelas = []
list_data = []

test_set = csv.reader(open('new_sefty.csv', 'r'), delimiter=',')
data_list = []
      

for index,data in enumerate(test_set):
    if index == 0:
        pass
    else:
        sentiment = data[1]
        tweet = data[2]
        tweet = processTweet(tweet)
        tweet = removeTagsAndLinks(tweet)
        tweet = getFeatureVector(tweet,stopWords)
        tweet = ' '.join(tweet)
        test_sent_features = {word.lower(): (word in word_tokenize(tweet.lower())) for word in all_words}
        prediksi = classifier.classify(test_sent_features)
        list_tweet.append(data[2])
        list_kelas.append(prediksi)
        mydata = (tweet,sentiment)
        data_list.append(mydata)

df = pd.DataFrame({'tweet': list_tweet,'kelas': list_kelas})
df.to_csv('hasiltest.csv', sep=';', encoding='utf-8', index=False)
        
test_set = [({word.lower() : (word.lower() in word_tokenize(x[0])) for word in all_words}, x[1]) for x in data_list]
accuracy = nltk.classify.util.accuracy(classifier, test_set)*100
print(accuracy)

40.400000000000006


In [17]:
feature = classifier.show_most_informative_features(50)

Most Informative Features
               menyentuh = True           negati : Positi =   1863.0 : 1.0
              tersandung = True           negati : Positi =   1863.0 : 1.0
                prihatin = True           negati : Positi =   1117.8 : 1.0
                   susah = True           negati : Positi =   1117.8 : 1.0
                   kepri = True           negati : Positi =    621.0 : 1.0
                kadernya = True           negati : Positi =    508.1 : 1.0
                    riau = True           negati : Negati =    490.1 : 1.0
                 positif = True           negati : Positi =    483.0 : 1.0
                     top = True           negati : Positi =    372.6 : 1.0
                  kontes = True           Positi : Positi =    285.0 : 1.0
                   domba = True           Positi : Negati =    283.6 : 1.0
                     bnn = True           negati : Negati =    272.3 : 1.0
                 edarkan = True           negati : Negati =    272.3 : 1.0